In [13]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout
#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
#from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.densenet import DenseNet201
#from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers.experimental.preprocessing import *
import numpy as np

In [2]:
img_width, img_height = (224,224)
batch_size = 32
class_names = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

train_dir = r"dataset_flowers\processed_data\train"
val_dir = r"dataset_flowers\processed_data\val"
test_dir = r"dataset_flowers\processed_data\test"

In [3]:
# create training data preprocessing generator
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=0.25,
    horizontal_flip=True)

# create training generator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    #classes=class_names,
    class_mode='categorical',
    subset='training')

# create validation generator
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    #classes=class_names,
    class_mode='categorical')

# create testing generator
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    #classes=class_names,
    class_mode='categorical')

# debug: draw sample test image and get shape
#x,y = test_generator.next()
#x.shape

Found 2593 images belonging to 5 classes.
Found 860 images belonging to 5 classes.
Found 865 images belonging to 5 classes.


In [17]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
#base_model.trainable = False
x = base_model.output
x = Dropout(0.1)(x)
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# mark precalculated base model layers as non-trainable; only our layers will remain trainable for future fitting
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_generator, epochs = 10, validation_data = val_generator) # use separate validation generator

Epoch 1/10
82/82 [==============================] - 26s 291ms/step - loss: 0.7422 - accuracy: 0.7335 - val_loss: 0.4227 - val_accuracy: 0.8523
Epoch 2/10
82/82 [==============================] - 23s 283ms/step - loss: 0.3594 - accuracy: 0.8739 - val_loss: 0.3348 - val_accuracy: 0.8791
Epoch 3/10
82/82 [==============================] - 23s 283ms/step - loss: 0.2936 - accuracy: 0.8936 - val_loss: 0.3498 - val_accuracy: 0.8698
Epoch 4/10
82/82 [==============================] - 23s 283ms/step - loss: 0.2429 - accuracy: 0.9202 - val_loss: 0.3109 - val_accuracy: 0.8919
Epoch 5/10
82/82 [==============================] - 23s 283ms/step - loss: 0.2358 - accuracy: 0.9113 - val_loss: 0.2925 - val_accuracy: 0.9047
Epoch 6/10
82/82 [==============================] - 23s 282ms/step - loss: 0.1865 - accuracy: 0.9383 - val_loss: 0.3137 - val_accuracy: 0.8837
Epoch 7/10
82/82 [==============================] - 23s 284ms/step - loss: 0.1819 - accuracy: 0.9425 - val_loss: 0.2762 - val_accuracy: 0.9035

In [19]:
test_result = model.evaluate(test_generator)
print("loss, accuracy: ", test_result)

865/865 [==============================] - 9s 11ms/step - loss: 0.2848 - accuracy: 0.9040
loss, accuracy:  [0.2848333418369293, 0.9040462374687195]


In [23]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [21]:
model.save('app/floweridentifymodel.h5')

In [22]:
import json

with open('app/floweridentifymodel.json', 'w') as outfile:
    json.dump(model.get_config(), outfile)

In [ ]:
# https://www.wildflower.org/image_archive/640x480/BL/BL_IMG0000106.JPG
# https://www.wildflower.org/plants-main/image-use-policy

In [20]:
img = tf.keras.preprocessing.image.load_img(
    "BL_IMG0000106.JPG", target_size=(224, 224)  # use same size as generated model
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.array([img_array])
#print(img_array.shape)
img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
predictions = model.predict(img_array, batch_size=1)
print(predictions)
score = tf.nn.softmax(predictions[0])
confidence = 100 * np.max(score)
#confidence = predictions[0][np.argmax(score)] * 100

print("{} ({:.2f} percent confident)".format(class_names[np.argmax(score)], confidence))


[[6.7043312e-02 4.5646592e-03 4.2704498e-05 9.2782730e-01 5.2211114e-04]]
sunflower (38.30 percent confident)
